In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os
import json
from bert_score import score
from tqdm import tqdm
# 设置可见GPU设备（根据实际GPU情况调整）
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 指定仅使用GPU 

# 路径配置 ------------------------------------------------------------------------
base_model_path = "/home/jzyoung/.cache/modelscope/hub/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B" # 原始预训练模型路径
peft_model_path = "./output/" # LoRA微调后保存的适配器路径

# 模型加载 ------------------------------------------------------------------------
# 初始化分词器（使用与训练时相同的tokenizer）
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 加载基础模型（半精度加载节省显存）
base_model = AutoModelForCausalLM.from_pretrained(
  base_model_path,
  torch_dtype=torch.float16, # 使用float16精度
  device_map="auto"      # 自动分配设备（CPU/GPU）
)

# 加载LoRA适配器（在基础模型上加载微调参数）
lora_model = PeftModel.from_pretrained(
  base_model, 
  peft_model_path,
  torch_dtype=torch.float16,
  device_map="auto"
)
# 合并LoRA权重到基础模型（提升推理速度，但会失去再次训练的能力）
lora_model = lora_model.merge_and_unload() 
lora_model.eval() # 设置为评估模式

# 生成函数 ------------------------------------------------------------------------
def generate_response(model, prompt):
  """统一的生成函数
  参数：
    model : 要使用的模型实例
    prompt : 符合格式要求的输入文本
  返回：
    清洗后的回答文本
  """
  # 输入编码（保持与训练时相同的处理方式）
  inputs = tokenizer(
    prompt,
    return_tensors="pt",     # 返回PyTorch张量
    max_length=1024,        # 最大输入长度（与训练时一致）
    truncation=True,       # 启用截断
    padding="max_length"     # 填充到最大长度（保证batch一致性）
  ).to(model.device)        # 确保输入与模型在同一设备

  # 文本生成（关闭梯度计算以节省内存）
  with torch.no_grad():
    outputs = model.generate(
      input_ids=inputs.input_ids,
      attention_mask=inputs.attention_mask,
      max_new_tokens=1024,    # 生成内容的最大token数（控制回答长度）
      temperature=0.7,     # 温度参数（0.0-1.0，值越大随机性越强）
      top_p=0.9,        # 核采样参数（保留累积概率前90%的token）
      repetition_penalty=1.1, # 重复惩罚系数（>1.0时抑制重复内容）
      eos_token_id=tokenizer.eos_token_id, # 结束符ID
      pad_token_id=tokenizer.pad_token_id, # 填充符ID 
    )
  
  # 解码与清洗输出
  full_text = tokenizer.decode(outputs[0], skip_special_tokens=True) # 跳过特殊token
  answer = full_text.split("### 答案：\n")[-1].strip() # 提取答案部分
  return answer

# 对比测试函数 --------------------------------------------------------------------
def compare_models(question):
  """模型对比函数
  参数：
    question : 自然语言形式的医疗问题
  """
  # 构建符合训练格式的prompt（注意与训练时格式完全一致）
  prompt = f"诊断问题：{question}\n详细分析：\n### 答案：\n"
  
  # 双模型生成
  base_answer = generate_response(base_model, prompt) # 原始模型
  lora_answer = generate_response(lora_model, prompt) # 微调模型
  
  # 终端彩色打印对比结果
  print("\n" + "="*50) # 分隔线
  print(f"问题：{question}")
  print("-"*50)
  print(f"\033[1;34m[原始模型]\033[0m\n{base_answer}") # 蓝色显示原始模型结果
  print("-"*50)
  print(f"\033[1;32m[LoRA模型]\033[0m\n{lora_answer}") # 绿色显示微调模型结果
  print("="*50 + "\n")


with open("./dataset/medical_o1_sft_Chinese.json") as f:
    test_data = json.load(f) 
    print("load file: ", test_data[:2])

# 数据量比较大，我们只选择10条数据进行测试
test_data=test_data[:10]
# 批量生成回答

def batch_generate(model, questions):
    answers = []
    for q in tqdm(questions):
        prompt = f"诊断问题：{q}\n详细分析：\n### 答案：\n"
        ans = generate_response(model, prompt)
        answers.append(ans)
    return answers

# 生成结果
print("Generating reponses of base model ...")
base_answers = batch_generate(base_model, [d["Question"] for d in test_data])
print("Generating reponses of lora sft model ...")
lora_answers = batch_generate(lora_model, [d["Question"] for d in test_data])
ref_answers = [d["Response"] for d in test_data]

bert_model_path="xxxxx/model/bert-base-chinese"
# 计算BERTScore
print("Comparing two responses ...")
_, _, base_bert = score(base_answers, ref_answers, lang="zh",model_type=bert_model_path,num_layers=12,device="cuda")
_, _, lora_bert = score(lora_answers, ref_answers, lang="zh",model_type=bert_model_path,num_layers=12,device="cuda")
print(f"BERTScore | 原始模型: {base_bert.mean().item():.3f} | LoRA模型: {lora_bert.mean().item():.3f}")


load file:  [{'Question': '根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？', 'Complex_CoT': '这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。', 'Response': '从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，

 20%|██        | 2/10 [00:22<01:26, 10.80s/it]